# Public administrative offices of Lecce (Puglia, Italy) & venues of interest in FourSquare

## Dario Di Marzo

## Table of Contents

* Introduction : The Business Problem
* Methodology : The Data
* Methodology : Data Cleaning
* Methodology : Data Visualization
* Methodology: Retrieve FourSquare Data
* Results: Map of venues always updated

## Introduction : The Business Problem

In this notebook we solve the problem to provide venues'informations for users visiting administrative offices in Lecce, Puglia. 
A user, specially tourist, so will enjoy a map, always updated by FourSquare, of places interest or private services in the proximity of administrative offices that he needs.
The stakeholders will be the public administration interested in a better public services development. 

## Methodology: The Data

[Foursquare page of Lecce venues](https://it.foursquare.com/explore?mode=url&ne=40.370874%2C18.212628&q=Cibi&sw=40.330515%2C18.140616)

[Open Data of Lecce Public Administration - Comune di Lecce](http://dati.comune.lecce.it/dataset/elenco-ubicazione-e-orari-sedi-comune-di-lecce)

In [89]:
# Import necessary libraries

import pandas as pd

print('libraries imported!')

libraries imported!


In [90]:
file='ComuneLecceGeo.csv'
df=pd.read_csv(file)
df.head(5)

,SETTORE,UFFICIO,AREA DI CIRCOLAZIONE,INDIRIZZO,NUMERO CIVICO,LAT,LON,ORARIO APERTURA PUBBLICO,UBICAZIONE
0,Affari Generali ed istituzionali - contenzioso...,Gabinetto del Sindaco,Via,Francesco Rubichi,16,40.35371,18.17155,non prevista,"Via Francesco Rubichi, 16"
1,Affari Generali ed istituzionali - contenzioso...,Gabinetto del Sindaco - Segreteria del Sindaco,Via,Francesco Rubichi,16,40.35371,18.17155,martedi 15.00 - 17.00 / venerdi 10.00 - 12.00,"Via Francesco Rubichi, 16"
2,Affari Generali ed istituzionali - contenzioso...,Gabinetto del Sindaco - Ufficio Comunicazione,Via,Francesco Rubichi,16,40.35371,18.17155,martedi 15.00 - 17.00 / venerdi 10.00 - 12.00,"Via Francesco Rubichi, 16"
3,Affari Generali ed istituzionali - contenzioso...,Gabinetto del Sindaco - Ufficio Stampa,Via,Francesco Rubichi,16,40.35371,18.17155,non prevista,"Via Francesco Rubichi, 16"
4,Affari Generali ed istituzionali - contenzioso...,Gabinetto del Sindaco - Ufficio del Cerimoniale,Via,Francesco Rubichi,16,40.35371,18.17155,dal lunedi al venerdi (previo appuntamento tel...,"Via Francesco Rubichi, 16"


Above a .csv file with a list of the public officies in Lecce and their Geo coordinates

## Methodology: Data Cleaning

Cutting of unecessary columns

In [91]:
pd.set_option('display.max_rows', 500)
df=df[['SETTORE','LAT','LON']]
df.head()

,SETTORE,LAT,LON
0,Affari Generali ed istituzionali - contenzioso...,40.35371,18.17155
1,Affari Generali ed istituzionali - contenzioso...,40.35371,18.17155
2,Affari Generali ed istituzionali - contenzioso...,40.35371,18.17155
3,Affari Generali ed istituzionali - contenzioso...,40.35371,18.17155
4,Affari Generali ed istituzionali - contenzioso...,40.35371,18.17155


Above we choice to keep column 'SETTORE' that groups different offices but with same purpose

Verifying data types

In [92]:
df.dtypes

SETTORE     object
LAT        float64
LON        float64
dtype: object

Dataframe shape before Nulls cut

In [93]:
df.shape[0]

162

Cutting eventually Nulls and shape after

In [94]:
df.dropna(inplace=True)
df.shape[0]

162

Counting unique values in SETTORE column

In [95]:
len(df['SETTORE'].value_counts())

14

We have offices of 14 different sectors

Counting unique values in LAT, LON columns

In [96]:
len(df['LAT'].value_counts())

24

In [97]:
len(df['LON'].value_counts())

24

Some offices for the same sector are in different buildings

So we keep only records with unique combination of LAT, LON

In [98]:
df.drop_duplicates(subset=['LAT', 'LON'], keep='last', inplace=True)
df.shape[0]

26

In [99]:
df['SETTORE'].value_counts()

Politiche Culturali, Valorizzazione Monumentale, Sviluppo Economico e Attività Produttive, Spettacolo, Sport, Turismo, Centro Storico e Archivio Storico    6
Servizi Demografici e Statistici e Servizi cimiteriali                                                                                                      4
Welfare, Casa e Pubblica Istruzione                                                                                                                         4
Affari Generali ed istituzionali - contenzioso - gare e appalti                                                                                             3
Pianificazione e sviluppo del territorio                                                                                                                    2
Lavori Pubblici  - Servizio edilizia scolastica - impiantistica sportiva                                                                                    2
Polizia Locale                                      

Above the different sectors in which Lecce administrative public office are grouped

## Methodology: Data Visualization

#### Create a map of Lecce with public offices superimposed on top.

Using library 'Geopy' to return coordinates of Lecce in Puglia, Italy

In [100]:
from geopy.geocoders import Nominatim

address = 'Lecce, IT'

geolocator = Nominatim(user_agent="le_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lecce are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lecce are 40.152217300000004, 18.2260628338229.


The coordinates returned are not exactly centerd on Lecce city. So i prefer to center the map on average coordinates from dataframe 

In [101]:
latitude=df['LAT'].mean()
longitude=df['LON'].mean()
print('The geograpical coordinate of Lecce are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lecce are 40.351757426090025, 18.177198750230854.


To create a map, to superimpose markers with names of public offices and to center this on Lecce coordinates i utilize Folium package

In [102]:
import folium
# matplotlib for colors
import matplotlib.cm as cm
import matplotlib.colors as colors

#requests for REST api
import requests

print('libraries imported!')

libraries imported!


In [103]:
# create map of Lecce using latitude and longitude values
map_lecce = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, office in zip(df['LAT'], df['LON'], df['SETTORE']):
    label = '{}'.format(office)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lecce)  
    
map_lecce

## Methodology: Retrieve FourSquare Data

#### Define Foursquare Credentials and Version

In [104]:
CLIENT_ID_ = 'XXX' # your Foursquare ID
CLIENT_SECRET_ = 'XXX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID_)
print('CLIENT_SECRET:' + CLIENT_SECRET_)

Your credentails:
CLIENT_ID: XXX
CLIENT_SECRET:XXX


#### a function to explore all the sectors

we fix radius of interest for FourSquare at 200

In [105]:
def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['SETTORE', 
                  'LAT', 
                  'LON', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [106]:
lecce_venues=getNearbyVenues(df['SETTORE'],df['LAT'],df['LON'])

In [107]:
lecce_venues.head(5)

,SETTORE,LAT,LON,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Affari Generali ed istituzionali - contenzioso...,40.35367,18.17219,Piazza Sant'Oronzo,40.353097,18.172734,Plaza
1,Affari Generali ed istituzionali - contenzioso...,40.35367,18.17219,Anfiteatro Romano,40.352500,18.172899,Historic Site
2,Affari Generali ed istituzionali - contenzioso...,40.35367,18.17219,Avio Bar,40.353517,18.173956,Coffee Shop
3,Affari Generali ed istituzionali - contenzioso...,40.35367,18.17219,Gelateria Natale,40.353413,18.173452,Ice Cream Shop
4,Affari Generali ed istituzionali - contenzioso...,40.35367,18.17219,Liberrima,40.352849,18.170449,Bookstore


Let's check how many venues were returned for each sector

In [108]:
lecce_venues.groupby('SETTORE').count()['Venue']

SETTORE
Affari Generali ed istituzionali - contenzioso - gare e appalti                                                                                             47
Ambiente e parchi - Igiene urbana e protezione civile                                                                                                        1
Innovazione tecnologica e agenda digitale                                                                                                                   13
Lavori Pubblici  - Servizio edilizia scolastica - impiantistica sportiva                                                                                     9
Pianificazione e sviluppo del territorio                                                                                                                     4
Politiche Culturali, Valorizzazione Monumentale, Sviluppo Economico e Attività Produttive, Spettacolo, Sport, Turismo, Centro Storico e Archivio Storico    88
Polizia Locale                        

How many unique category of venue

In [109]:
print('There are {} uniques categories.'.format(len(lecce_venues['Venue Category'].unique())))

There are 37 uniques categories.


Now we can superimpose markers of venues from FourSquare on map

In [110]:
# add markers to map
for lat, lng, office in zip(lecce_venues['Venue Latitude'], lecce_venues['Venue Longitude'], lecce_venues['Venue']):
    label = '{}'.format(office)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc318e',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lecce)  
    
map_lecce

## Results: Map of venues always updated

Thanks FourSquare a tourist or citizen can check for a venue of interest when he may need to visit a pubblic administrative office in Lecce